In [1]:
import numpy as np
import pandas as pd

In [2]:
temp_df = pd.read_csv("C:\\Users\\Saurav\\Downloads\\IMDB Dataset.csv")

In [4]:
df = temp_df.iloc[:10000]

In [5]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [7]:
df.duplicated().sum()

17

In [8]:
df.drop_duplicates(inplace=True)

C:\Users\Saurav\AppData\Local\Temp\ipykernel_5188\3006716147.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)


In [9]:
df.duplicated().sum()

0

In [14]:
## remove the html tags
import re
def remove_tags(raw_text):
    cleaned_text = re.sub(re.compile('<.*?>') , '', raw_text)
    return cleaned_text

In [15]:
df['review'] =  df['review'].apply(remove_tags)

C:\Users\Saurav\AppData\Local\Temp\ipykernel_5188\623568053.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] =  df['review'].apply(remove_tags)


In [16]:
df['review'][1]

'A wonderful little production. The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. The actors are extremely well chosen- Michael Sheen not only "has got all the polari" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams\' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master\'s of comedy and his life. The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional \'dream\' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell\'s murals decorating every surface) are terribly well done.'

In [17]:
df['review']= df['review'].apply(lambda x:x.lower())

C:\Users\Saurav\AppData\Local\Temp\ipykernel_5188\3441098086.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review']= df['review'].apply(lambda x:x.lower())


In [18]:
df['review'][1]

'a wonderful little production. the filming technique is very unassuming- very old-time-bbc fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. the actors are extremely well chosen- michael sheen not only "has got all the polari" but he has all the voices down pat too! you can truly see the seamless editing guided by the references to williams\' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. a masterful production about one of the great master\'s of comedy and his life. the realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional \'dream\' techniques remains solid then disappears. it plays on our knowledge and our senses, particularly with the scenes concerning orton and halliwell and the sets (particularly of their flat with halliwell\'s murals decorating every surface) are terribly well done.'

In [19]:
from nltk.corpus import stopwords

sw_list = stopwords.words('english')

df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))

C:\Users\Saurav\AppData\Local\Temp\ipykernel_5188\2826946130.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))


In [20]:
df['review']

0       one reviewers mentioned watching 1 oz episode ...
1       wonderful little production. filming technique...
2       thought wonderful way spend time hot summer we...
3       basically there's family little boy (jake) thi...
4       petter mattei's "love time money" visually stu...
                              ...                        
9995    fun, entertaining movie wwii german spy (julie...
9996    give break. anyone say "good hockey movie"? kn...
9997    movie bad movie. watching endless series bad h...
9998    movie probably made entertain middle school, e...
9999    smashing film film-making. shows intense stran...
Name: review, Length: 9983, dtype: object

In [21]:
import gensim

In [22]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [24]:
story = []
for doc in df['review']:
    raw_sent = sent_tokenize(doc)
    for sent in raw_sent:
        story.append(simple_preprocess(sent))

In [29]:
model = gensim.models.Word2Vec(
    window= 10, 
    min_count= 2
)

In [30]:
model.build_vocab(story)

In [32]:
model.train(story, total_examples=model.corpus_count, epochs=model.epochs)

(5850032, 6186875)

In [33]:
len(model.wv.index_to_key)

31845

In [34]:
def document_vector(doc):
    # remove out-of-vocabulary words
    doc = [word for word in doc.split() if word in model.wv.index_to_key]
    return np.mean(model.wv[doc], axis=0)

In [35]:
document_vector(df['review'].values[0])

array([ 0.03526195,  0.330026  ,  0.23091309,  0.00702401, -0.05199333,
       -0.53868663,  0.15321092,  0.8305338 , -0.17743486, -0.22606032,
       -0.21090455, -0.53616154, -0.0536106 ,  0.2766935 ,  0.20398928,
       -0.24820116,  0.1641319 , -0.5679592 , -0.08185983, -0.6514167 ,
       -0.00713135,  0.15505603,  0.21121469, -0.24316365, -0.1868334 ,
       -0.13165568, -0.15469272, -0.23030199, -0.3750689 , -0.03857365,
        0.39227372, -0.01161173, -0.07972447, -0.21247296, -0.4264797 ,
        0.37356862,  0.05280544, -0.29887477, -0.13329788, -0.72688925,
        0.15032399, -0.22744581, -0.17392324,  0.00189253,  0.41629735,
       -0.13469607, -0.45936352, -0.22447853,  0.08889364,  0.3093023 ,
        0.13690737, -0.3156131 , -0.36115208, -0.17296833, -0.38368863,
        0.06408819,  0.3901138 ,  0.19666189, -0.28370446,  0.0796501 ,
       -0.05679084,  0.14962883, -0.20486264, -0.00518711, -0.37346783,
        0.4462726 , -0.03156149,  0.07596012, -0.5692992 ,  0.33

In [36]:
from tqdm import tqdm

In [37]:
X = []
for doc in tqdm(df['review'].values):
    X.append(document_vector(doc))

100%|██████████████████████████████████████████████████████████████████████████████| 9983/9983 [30:52<00:00,  5.39it/s]


In [42]:
X = np.array(X)

In [45]:
X[0]

array([ 0.03526195,  0.330026  ,  0.23091309,  0.00702401, -0.05199333,
       -0.53868663,  0.15321092,  0.8305338 , -0.17743486, -0.22606032,
       -0.21090455, -0.53616154, -0.0536106 ,  0.2766935 ,  0.20398928,
       -0.24820116,  0.1641319 , -0.5679592 , -0.08185983, -0.6514167 ,
       -0.00713135,  0.15505603,  0.21121469, -0.24316365, -0.1868334 ,
       -0.13165568, -0.15469272, -0.23030199, -0.3750689 , -0.03857365,
        0.39227372, -0.01161173, -0.07972447, -0.21247296, -0.4264797 ,
        0.37356862,  0.05280544, -0.29887477, -0.13329788, -0.72688925,
        0.15032399, -0.22744581, -0.17392324,  0.00189253,  0.41629735,
       -0.13469607, -0.45936352, -0.22447853,  0.08889364,  0.3093023 ,
        0.13690737, -0.3156131 , -0.36115208, -0.17296833, -0.38368863,
        0.06408819,  0.3901138 ,  0.19666189, -0.28370446,  0.0796501 ,
       -0.05679084,  0.14962883, -0.20486264, -0.00518711, -0.37346783,
        0.4462726 , -0.03156149,  0.07596012, -0.5692992 ,  0.33

In [46]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

y = encoder.fit_transform(df['sentiment'])

In [47]:
y

array([1, 1, 1, ..., 0, 0, 1])

In [48]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [49]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [50]:
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.785177766649975